# uFactory Lite 6 Robot Control Notebook

This notebook provides interactive control of the uFactory Lite 6 robot arm with:
- Real-time status monitoring
- Individual joint control via sliders
- Safety features (low speed limits, confirmation buttons)
- Live position feedback

## Safety Notes
⚠️ **IMPORTANT**:
- Ensure the robot workspace is clear before running commands
- Default speeds are set very low (5 deg/s) for safety
- Always monitor the robot during operation
- Use emergency stop if needed (Ctrl+C or stop button)

## 1. Setup and Imports

In [ ]:
# Install required packages (run once)
# !pip install xarm-python-sdk ipywidgets pandas numpy

In [ ]:
import time
import numpy as np
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

try:
    from xarm.wrapper import XArmAPI
    print("✓ xArm SDK imported successfully")
except ImportError:
    print("✗ xArm SDK not found. Install with: pip install xarm-python-sdk")
    raise

## 2. Configuration and Connection

In [ ]:
# Robot configuration
ROBOT_IP = "192.168.1.161"  # ⚠️ CHANGE THIS TO YOUR ROBOT'S IP ADDRESS

# Safety limits
DEFAULT_SPEED = 5.0  # deg/s - Very slow for safety
MAX_SPEED = 30.0     # deg/s - Maximum allowed speed
MIN_SPEED = 1.0      # deg/s - Minimum speed

# Joint angle limits for Lite6 (degrees)
JOINT_LIMITS = {
    'J1': (-360, 360),
    'J2': (-118, 120),
    'J3': (-225, 11),
    'J4': (-360, 360),
    'J5': (-97, 180),
    'J6': (-360, 360)
}

print(f"Configuration:")
print(f"  Robot IP: {ROBOT_IP}")
print(f"  Default Speed: {DEFAULT_SPEED} deg/s")
print(f"  Max Speed: {MAX_SPEED} deg/s")

In [ ]:
# Connect to robot
def connect_robot(ip):
    """Connect and initialize the robot."""
    try:
        print(f"Connecting to robot at {ip}...")
        arm = XArmAPI(ip)
        
        # Clean any previous errors
        arm.clean_error()
        print("✓ Cleared previous errors")
        
        # Enable motion
        arm.motion_enable(enable=True)
        print("✓ Motion enabled")
        
        # Set to position control mode (mode 0)
        arm.set_mode(0)
        print("✓ Set to position control mode")
        
        # Set state to ready (state 0)
        arm.set_state(state=0)
        print("✓ Robot state set to ready")
        
        print("\n✓ Robot connected and initialized successfully!")
        return arm
    
    except Exception as e:
        print(f"✗ Connection failed: {e}")
        raise

# Connect
arm = connect_robot(ROBOT_IP)

## 3. Status Display Functions

In [ ]:
def get_robot_status(arm):
    """Get comprehensive robot status."""
    status = {}
    
    # Joint angles
    code, angles = arm.get_servo_angle()
    if code == 0:
        status['joint_angles'] = angles
    
    # Cartesian position
    code, pose = arm.get_position()
    if code == 0:
        status['cartesian_pose'] = pose
    
    # Robot state
    code, state = arm.get_state()
    if code == 0:
        status['state'] = state
    
    # Error/warning codes
    code, err_warn = arm.get_err_warn_code()
    if code == 0:
        status['error_code'] = err_warn[0] if len(err_warn) > 0 else 0
        status['warning_code'] = err_warn[1] if len(err_warn) > 1 else 0
    
    # Mode
    status['mode'] = arm.mode
    
    return status

def display_status(arm):
    """Display formatted robot status."""
    status = get_robot_status(arm)
    
    # Create HTML display
    html = "<h3>Robot Status</h3>"
    
    # Joint angles
    if 'joint_angles' in status:
        angles = status['joint_angles']
        html += "<h4>Joint Angles (degrees)</h4>"
        html += "<table style='width:100%; border-collapse: collapse;'>"
        html += "<tr style='background-color: #f0f0f0;'>"
        for i in range(6):
            html += f"<th style='border: 1px solid #ddd; padding: 8px;'>J{i+1}</th>"
        html += "</tr><tr>"
        for angle in angles:
            html += f"<td style='border: 1px solid #ddd; padding: 8px; text-align: center;'>{angle:.2f}°</td>"
        html += "</tr></table><br>"
    
    # Cartesian position
    if 'cartesian_pose' in status:
        pose = status['cartesian_pose']
        html += "<h4>Cartesian Pose</h4>"
        html += f"<p><b>Position (mm):</b> X={pose[0]:.1f}, Y={pose[1]:.1f}, Z={pose[2]:.1f}</p>"
        html += f"<p><b>Rotation (deg):</b> Roll={pose[3]:.1f}, Pitch={pose[4]:.1f}, Yaw={pose[5]:.1f}</p>"
    
    # State and mode
    state_names = {0: 'Ready', 1: 'Pause', 3: 'Offline', 4: 'Error', 5: 'Stop'}
    mode_names = {0: 'Position', 1: 'Servo', 2: 'Joint Teaching', 4: 'Cartesian Teaching'}
    
    if 'state' in status:
        state_str = state_names.get(status['state'], f'Unknown ({status["state"]})')
        color = 'green' if status['state'] == 0 else 'orange'
        html += f"<p><b>State:</b> <span style='color: {color};'>{state_str}</span></p>"
    
    if 'mode' in status:
        mode_str = mode_names.get(status['mode'], f'Unknown ({status["mode"]})')
        html += f"<p><b>Mode:</b> {mode_str}</p>"
    
    # Errors
    if 'error_code' in status:
        if status['error_code'] != 0:
            html += f"<p style='color: red;'><b>Error Code:</b> {status['error_code']}</p>"
        else:
            html += "<p style='color: green;'><b>No errors</b></p>"
    
    if 'warning_code' in status and status['warning_code'] != 0:
        html += f"<p style='color: orange;'><b>Warning Code:</b> {status['warning_code']}</p>"
    
    display(HTML(html))

# Display current status
display_status(arm)

## 4. Quick Status Check (Run Anytime)

In [ ]:
# Quick status refresh - run this cell anytime to check current position
display_status(arm)

## 5. Basic Movement Commands

In [ ]:
def move_to_joint_angles(arm, angles, speed=DEFAULT_SPEED, wait=True):
    """Move to specified joint angles with safety checks."""
    # Clamp speed to safe range
    speed = max(MIN_SPEED, min(speed, MAX_SPEED))
    
    print(f"Moving to: {[f'{a:.2f}' for a in angles]} at {speed} deg/s")
    
    try:
        code = arm.set_servo_angle(angle=angles, speed=speed, wait=wait, is_radian=False)
        if code == 0:
            print("✓ Movement completed successfully")
            return True
        else:
            print(f"✗ Movement failed with code: {code}")
            return False
    except Exception as e:
        print(f"✗ Movement error: {e}")
        return False

def go_home(arm, speed=DEFAULT_SPEED):
    """Move to home position (all joints at 0 degrees)."""
    print("Moving to home position...")
    return move_to_joint_angles(arm, [0, 0, 0, 0, 0, 0], speed=speed)

In [ ]:
# Move to home position
go_home(arm, speed=10.0)

## 6. Interactive Joint Control with Sliders

Use the sliders below to adjust individual joints. The robot will move when you click "Move Robot".

In [ ]:
# Get current joint angles as starting point
code, current_angles = arm.get_servo_angle()
if code != 0:
    current_angles = [0, 0, 0, 0, 0, 0]

# Create sliders for each joint
joint_sliders = {}
slider_layout = widgets.Layout(width='500px')

for i in range(6):
    joint_name = f'J{i+1}'
    min_val, max_val = JOINT_LIMITS[joint_name]
    
    joint_sliders[joint_name] = widgets.FloatSlider(
        value=current_angles[i],
        min=min_val,
        max=max_val,
        step=0.5,
        description=f'{joint_name}:',
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        layout=slider_layout,
        style={'description_width': '60px'}
    )

# Speed slider
speed_slider = widgets.FloatSlider(
    value=DEFAULT_SPEED,
    min=MIN_SPEED,
    max=MAX_SPEED,
    step=0.5,
    description='Speed (deg/s):',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=slider_layout,
    style={'description_width': '120px'}
)

# Output area
output = widgets.Output()

# Buttons
move_button = widgets.Button(
    description='Move Robot',
    button_style='success',
    tooltip='Send joint angles to robot',
    icon='play'
)

update_button = widgets.Button(
    description='Update from Robot',
    button_style='info',
    tooltip='Read current position from robot',
    icon='refresh'
)

home_button = widgets.Button(
    description='Go Home',
    button_style='warning',
    tooltip='Move to home position',
    icon='home'
)

stop_button = widgets.Button(
    description='STOP',
    button_style='danger',
    tooltip='Emergency stop',
    icon='stop'
)

# Button callbacks
def on_move_clicked(b):
    with output:
        clear_output()
        angles = [joint_sliders[f'J{i+1}'].value for i in range(6)]
        speed = speed_slider.value
        move_to_joint_angles(arm, angles, speed=speed)
        display_status(arm)

def on_update_clicked(b):
    with output:
        clear_output()
        code, angles = arm.get_servo_angle()
        if code == 0:
            for i in range(6):
                joint_sliders[f'J{i+1}'].value = angles[i]
            print("✓ Updated sliders from robot position")
        else:
            print(f"✗ Failed to read position (code: {code})")
        display_status(arm)

def on_home_clicked(b):
    with output:
        clear_output()
        speed = speed_slider.value
        if go_home(arm, speed=speed):
            for i in range(6):
                joint_sliders[f'J{i+1}'].value = 0.0
        display_status(arm)

def on_stop_clicked(b):
    with output:
        clear_output()
        print("⚠️ EMERGENCY STOP")
        try:
            arm.emergency_stop()
            print("✓ Robot stopped")
        except:
            arm.set_state(4)  # Stop state
            print("✓ Robot stopped (via set_state)")

move_button.on_click(on_move_clicked)
update_button.on_click(on_update_clicked)
home_button.on_click(on_home_clicked)
stop_button.on_click(on_stop_clicked)

# Layout
print("Interactive Joint Control")
print("="*60)
print("Adjust the sliders below and click 'Move Robot' to execute.")
print("⚠️ Start with small movements and low speed!")
print()

# Display all controls
display(widgets.VBox([
    widgets.HTML("<h3>Joint Angles</h3>"),
    joint_sliders['J1'],
    joint_sliders['J2'],
    joint_sliders['J3'],
    joint_sliders['J4'],
    joint_sliders['J5'],
    joint_sliders['J6'],
    widgets.HTML("<h3>Speed Control</h3>"),
    speed_slider,
    widgets.HTML("<h3>Controls</h3>"),
    widgets.HBox([move_button, update_button, home_button, stop_button]),
    widgets.HTML("<h3>Output</h3>"),
    output
]))

## 7. Fine-Tuning Individual Joints

Use these controls to make very small adjustments to individual joints.

In [ ]:
# Single joint fine control
def create_single_joint_control():
    # Get current position
    code, current_angles = arm.get_servo_angle()
    if code != 0:
        current_angles = [0, 0, 0, 0, 0, 0]
    
    # Joint selector
    joint_selector = widgets.Dropdown(
        options=['J1', 'J2', 'J3', 'J4', 'J5', 'J6'],
        value='J1',
        description='Joint:',
        style={'description_width': '80px'}
    )
    
    # Delta adjustment slider (small increments)
    delta_slider = widgets.FloatSlider(
        value=0.0,
        min=-10.0,
        max=10.0,
        step=0.1,
        description='Adjust (deg):',
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='500px'),
        style={'description_width': '100px'}
    )
    
    # Current value display
    current_value = widgets.FloatText(
        value=current_angles[0],
        description='Current:',
        disabled=True,
        style={'description_width': '80px'}
    )
    
    # Target value display
    target_value = widgets.FloatText(
        value=current_angles[0],
        description='Target:',
        disabled=True,
        style={'description_width': '80px'}
    )
    
    # Fine speed control
    fine_speed = widgets.FloatSlider(
        value=3.0,
        min=1.0,
        max=10.0,
        step=0.5,
        description='Speed (deg/s):',
        continuous_update=False,
        layout=widgets.Layout(width='500px'),
        style={'description_width': '100px'}
    )
    
    output2 = widgets.Output()
    
    # Update displays when joint or delta changes
    def update_displays(*args):
        joint_idx = int(joint_selector.value[1]) - 1
        code, angles = arm.get_servo_angle()
        if code == 0:
            current = angles[joint_idx]
            current_value.value = current
            target_value.value = current + delta_slider.value
    
    joint_selector.observe(update_displays, 'value')
    delta_slider.observe(update_displays, 'value')
    
    # Apply button
    apply_button = widgets.Button(
        description='Apply Adjustment',
        button_style='success',
        icon='check'
    )
    
    reset_button = widgets.Button(
        description='Reset Delta',
        button_style='warning',
        icon='undo'
    )
    
    def on_apply_clicked(b):
        with output2:
            clear_output()
            joint_idx = int(joint_selector.value[1]) - 1
            
            # Get current angles
            code, angles = arm.get_servo_angle()
            if code != 0:
                print("✗ Failed to read current position")
                return
            
            # Apply delta to selected joint
            new_angles = list(angles)
            new_angles[joint_idx] += delta_slider.value
            
            # Check limits
            joint_name = joint_selector.value
            min_val, max_val = JOINT_LIMITS[joint_name]
            if new_angles[joint_idx] < min_val or new_angles[joint_idx] > max_val:
                print(f"✗ Target angle {new_angles[joint_idx]:.1f}° is outside limits [{min_val}, {max_val}]")
                return
            
            print(f"{joint_name}: {angles[joint_idx]:.2f}° → {new_angles[joint_idx]:.2f}° (Δ={delta_slider.value:.2f}°)")
            move_to_joint_angles(arm, new_angles, speed=fine_speed.value)
            
            # Reset delta and update displays
            delta_slider.value = 0.0
            update_displays()
    
    def on_reset_clicked(b):
        delta_slider.value = 0.0
        update_displays()
    
    apply_button.on_click(on_apply_clicked)
    reset_button.on_click(on_reset_clicked)
    
    # Initial update
    update_displays()
    
    # Display
    display(widgets.VBox([
        widgets.HTML("<h3>Fine Joint Control</h3>"),
        widgets.HTML("<p>Select a joint and adjust it by small increments.</p>"),
        joint_selector,
        widgets.HBox([current_value, target_value]),
        delta_slider,
        fine_speed,
        widgets.HBox([apply_button, reset_button]),
        output2
    ]))

create_single_joint_control()

## 8. Save and Load Positions

In [ ]:
# Dictionary to store saved positions
saved_positions = {}

def save_current_position(name):
    """Save the current robot position."""
    code, angles = arm.get_servo_angle()
    if code == 0:
        saved_positions[name] = angles
        print(f"✓ Saved position '{name}': {[f'{a:.2f}' for a in angles]}")
        return True
    else:
        print(f"✗ Failed to read position")
        return False

def load_position(name, speed=DEFAULT_SPEED):
    """Load and move to a saved position."""
    if name in saved_positions:
        angles = saved_positions[name]
        print(f"Loading position '{name}': {[f'{a:.2f}' for a in angles]}")
        return move_to_joint_angles(arm, angles, speed=speed)
    else:
        print(f"✗ Position '{name}' not found")
        return False

def list_saved_positions():
    """List all saved positions."""
    if not saved_positions:
        print("No saved positions")
        return
    
    print("Saved positions:")
    for name, angles in saved_positions.items():
        print(f"  {name}: {[f'{a:.2f}' for a in angles]}")

# Example usage
# save_current_position('home')
# load_position('home')
# list_saved_positions()

In [ ]:
# Interactive position manager
position_name_input = widgets.Text(
    value='',
    placeholder='Enter position name',
    description='Name:',
    style={'description_width': '80px'}
)

position_selector = widgets.Dropdown(
    options=list(saved_positions.keys()),
    description='Load:',
    style={'description_width': '80px'}
)

save_pos_button = widgets.Button(description='Save Current', button_style='info', icon='save')
load_pos_button = widgets.Button(description='Load Selected', button_style='success', icon='upload')
list_pos_button = widgets.Button(description='List All', button_style='', icon='list')

output3 = widgets.Output()

def on_save_pos_clicked(b):
    with output3:
        clear_output()
        if position_name_input.value:
            save_current_position(position_name_input.value)
            position_selector.options = list(saved_positions.keys())
        else:
            print("✗ Please enter a position name")

def on_load_pos_clicked(b):
    with output3:
        clear_output()
        if position_selector.value:
            load_position(position_selector.value, speed=DEFAULT_SPEED)

def on_list_pos_clicked(b):
    with output3:
        clear_output()
        list_saved_positions()

save_pos_button.on_click(on_save_pos_clicked)
load_pos_button.on_click(on_load_pos_clicked)
list_pos_button.on_click(on_list_pos_clicked)

display(widgets.VBox([
    widgets.HTML("<h3>Position Manager</h3>"),
    widgets.HBox([position_name_input, save_pos_button]),
    widgets.HBox([position_selector, load_pos_button, list_pos_button]),
    output3
]))

## 9. Trajectory Recording and Playback

In [ ]:
import json
from pathlib import Path

# Trajectory recording
recorded_trajectory = []

def record_waypoint():
    """Record current position as a waypoint."""
    code, angles = arm.get_servo_angle()
    if code == 0:
        recorded_trajectory.append(angles)
        print(f"✓ Recorded waypoint {len(recorded_trajectory)}: {[f'{a:.2f}' for a in angles]}")
        return True
    else:
        print("✗ Failed to record waypoint")
        return False

def clear_trajectory():
    """Clear recorded trajectory."""
    global recorded_trajectory
    recorded_trajectory = []
    print("✓ Trajectory cleared")

def play_trajectory(speed=DEFAULT_SPEED, delay=0.5):
    """Play back recorded trajectory."""
    if not recorded_trajectory:
        print("✗ No trajectory recorded")
        return False
    
    print(f"Playing trajectory with {len(recorded_trajectory)} waypoints...")
    for i, angles in enumerate(recorded_trajectory):
        print(f"Waypoint {i+1}/{len(recorded_trajectory)}")
        if not move_to_joint_angles(arm, angles, speed=speed, wait=True):
            print("✗ Trajectory stopped due to error")
            return False
        time.sleep(delay)
    
    print("✓ Trajectory playback complete")
    return True

def save_trajectory(filename):
    """Save trajectory to JSON file."""
    if not recorded_trajectory:
        print("✗ No trajectory to save")
        return False
    
    data = {"joints": recorded_trajectory}
    path = Path(filename)
    path.write_text(json.dumps(data, indent=2))
    print(f"✓ Saved trajectory to {filename}")
    return True

def load_trajectory(filename):
    """Load trajectory from JSON file."""
    global recorded_trajectory
    
    path = Path(filename)
    if not path.exists():
        print(f"✗ File not found: {filename}")
        return False
    
    data = json.loads(path.read_text())
    if "joints" in data:
        recorded_trajectory = data["joints"]
    elif "trajectory" in data:
        recorded_trajectory = data["trajectory"]
    else:
        print("✗ Invalid trajectory file format")
        return False
    
    print(f"✓ Loaded trajectory with {len(recorded_trajectory)} waypoints from {filename}")
    return True

In [ ]:
# Trajectory controls
traj_filename = widgets.Text(
    value='trajectory.json',
    description='Filename:',
    style={'description_width': '80px'}
)

traj_speed = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=20.0,
    step=1.0,
    description='Play Speed:',
    style={'description_width': '80px'}
)

traj_delay = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=2.0,
    step=0.1,
    description='Delay (s):',
    style={'description_width': '80px'}
)

record_button = widgets.Button(description='Record Waypoint', button_style='info', icon='circle')
clear_traj_button = widgets.Button(description='Clear', button_style='warning', icon='trash')
play_button = widgets.Button(description='Play', button_style='success', icon='play')
save_traj_button = widgets.Button(description='Save', button_style='', icon='save')
load_traj_button = widgets.Button(description='Load', button_style='', icon='upload')

output4 = widgets.Output()

def on_record_clicked(b):
    with output4:
        clear_output()
        record_waypoint()

def on_clear_traj_clicked(b):
    with output4:
        clear_output()
        clear_trajectory()

def on_play_clicked(b):
    with output4:
        clear_output()
        play_trajectory(speed=traj_speed.value, delay=traj_delay.value)

def on_save_traj_clicked(b):
    with output4:
        clear_output()
        save_trajectory(traj_filename.value)

def on_load_traj_clicked(b):
    with output4:
        clear_output()
        load_trajectory(traj_filename.value)

record_button.on_click(on_record_clicked)
clear_traj_button.on_click(on_clear_traj_clicked)
play_button.on_click(on_play_clicked)
save_traj_button.on_click(on_save_traj_clicked)
load_traj_button.on_click(on_load_traj_clicked)

display(widgets.VBox([
    widgets.HTML("<h3>Trajectory Recording & Playback</h3>"),
    widgets.HTML("<p>1. Move robot to desired positions<br>2. Click 'Record Waypoint' at each position<br>3. Click 'Play' to replay the trajectory</p>"),
    widgets.HBox([record_button, clear_traj_button, play_button]),
    traj_speed,
    traj_delay,
    widgets.HTML("<h4>Save/Load</h4>"),
    traj_filename,
    widgets.HBox([save_traj_button, load_traj_button]),
    output4
]))

## 10. Cleanup and Disconnect

In [ ]:
# Disconnect from robot
def disconnect_robot(arm):
    """Safely disconnect from robot."""
    try:
        print("Disconnecting from robot...")
        arm.disconnect()
        print("✓ Disconnected successfully")
    except Exception as e:
        print(f"Error during disconnect: {e}")

# Uncomment to disconnect:
# disconnect_robot(arm)

## 11. Utilities

In [ ]:
# Clear errors and re-initialize
def reset_robot(arm):
    """Clear errors and re-initialize robot."""
    try:
        print("Resetting robot...")
        arm.clean_error()
        arm.motion_enable(enable=True)
        arm.set_mode(0)
        arm.set_state(state=0)
        print("✓ Robot reset successfully")
        display_status(arm)
    except Exception as e:
        print(f"✗ Reset failed: {e}")

# Uncomment to reset:
# reset_robot(arm)